In [1]:
import pandas as pd
import unidecode
import os

In [2]:
path_municipio_base ='..\\..\\Bases\\Municipios\\'
file_municipios = 'Municipios_geo_clima.csv'

dir_bases = path_municipio_base
os.chdir(dir_bases)
os.getcwd()

'G:\\Users\\johns\\Desktop\\TCC\\MPAlugueisLisboa\\Bases\\Municipios'

In [3]:
# Carrega Base Principal
file_path = '{}{}'.format(path_municipio_base,file_municipios)
df_municipios = pd.read_csv(file_path, sep=";", decimal=',', encoding='utf-8')

# atualiza para unicode os campos
df_municipios['Regiao'] = df_municipios['Regiao'].apply(lambda x: unidecode.unidecode(x)) 

In [4]:
# Filtra colunas
lst_municipios_columns = ['Ambito Geografico',
       'Regiao', 'Cidades', 'qtd Idade 0 -14', 'qtd Idade 15-64',
       'qtd idade >65', '% Idade 0 -14', '% Idade 15-64', '% idade >65',
       'N.º médio de indivíduos por Km²\t\n\t',
       '% cidadãos estrangeiros,em relacao total da população',
       'qtd Hospitais particulares e publicos', 'qtd Farmacias',
       'Quantidade construçoes novas', 'qtd escolas - pre-escolar -4 -6 anos',
       'qtd escolas - ciclo 1 - 6-10 anos',
       'qtd escolas - ciclo 2 - 10 -12 anos',
       'qtd escolas - ciclo 3 - 13 -15 anos', 'qtd shows,exibicoes,teatro',
       'qtd Cinemas', 'Ganho medio mensal', '% Desempregados',
       '% Poder de compra', '% Crimes', 'qtd bancos', 'Latitude', 'Longitude',
       'Address', 'distance_driving_Lisboa_Portugal', 'distance_driving_Porto_Portugal', 'Clima_Ranking']
df_municipios = df_municipios.filter(lst_municipios_columns)


# Define variaveis que serão usadas na analise do ranking
lst_analise = ['qtd Idade 15-64',
 '% Idade 0 -14',
 'qtd Hospitais particulares e publicos',
 'qtd Farmacias',
 'Quantidade construçoes novas',
 'qtd escolas - pre-escolar -4 -6 anos',
 'qtd escolas - ciclo 1 - 6-10 anos',
 'qtd shows,exibicoes,teatro',
 'qtd Cinemas',
 'Ganho medio mensal',
 '% Desempregados',
 '% Poder de compra',
 '% Crimes',
 'qtd bancos']


In [17]:
def calcula_ranking(list_var, df, tipo_ranking = 'Ranking_Geral'):
    
    # Por Variavel pega os valores hight, mean e low
    for var in list_var:
        # Cria Prefixo para identificar colunas gerados pelas variaveis analizadas
        prefix = 'GRESULT_'
        new_col = prefix + var
        new_col = new_col.replace(" ", "_")
        df[new_col] = 0
        
        # aplica a função à coluna "Assaltos por Mês" e cria a nova coluna "Novo Ranking"
        df[new_col] = df[var]
        
        # aplica peso negativo para variaveis negativas
        if new_col in ['GRESULT_%_Desempregados','GRESULT_%_Crimes']:
            df[new_col] = new_col * -1
            
    # cria lista com os campos criados por variavel
    lst_columns = [columns for columns in df if prefix in columns]

    # soma os valores dos campos(colunas) para criar coluna de ranking
    df['GRESULT_Ranking_Geral'] = df[lst_columns].sum(axis=1)
    
    df['Ranking_Geral'] = df['GRESULT_Ranking_Geral'].rank(method='dense', ascending=False)
    df['Ranking_Regional'] = df.groupby('Regiao')['GRESULT_Ranking_Geral'].rank(method='dense', ascending=False)
    
    #exclui campos gerados pelas variaveis analizadas
    prefix = 'GRESULT_'
    lst_columns = [columns for columns in df if prefix in columns]
    df.drop(columns = lst_columns, inplace=True)
    


In [18]:
calcula_ranking(lst_analise, df_municipios)

C:\Users\johns\AppData\Local\Temp\ipykernel_17412\986127812.py:22: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df['GRESULT_Ranking_Geral'] = df[lst_columns].sum(axis=1)


,Ambito Geografico,Regiao,Cidades,qtd Idade 0 -14,qtd Idade 15-64,qtd idade >65,% Idade 0 -14,% Idade 15-64,% idade >65,N.º médio de indivíduos por Km²\t\n\t,...,% Crimes,qtd bancos,Latitude,Longitude,Address,distance_driving_Lisboa_Portugal,distance_driving_Porto_Portugal,Clima_Ranking,Ranking_Geral,Ranking_Regional
0,Município,Norte,Arcos de Valdevez,1988,12099,6770,9.5,58.0,32.5,46.6,...,29.2,6,41.847198,-8.417882,"Arcos de Valdevez, Viana do Castelo, Portugal",353.9,79.2,18.0,127.0,43.0
1,Município,Norte,Caminha,1736,9981,4115,11.0,63.0,26.0,116.0,...,29.2,8,41.875760,-8.838838,"Caminha, Viana do Castelo, Portugal",352.7,82.9,13.0,144.0,49.0
2,Município,Norte,Melgaco,630,4505,2908,7.8,56.0,36.2,33.8,...,26.4,5,42.112909,-8.258067,"Melgaço, Vila e Roussas, Melgaço, Viana do Cas...",385.4,111.0,12.0,214.0,66.0
3,Município,Norte,Moncao,1693,10844,5262,9.5,60.9,29.6,84.2,...,17.8,6,42.027879,-8.417329,"Monção, Viana do Castelo, Portugal",373.7,98.9,13.0,134.0,45.0
4,Município,Norte,Paredes de Coura,942,5186,2376,11.1,61.0,27.9,61.5,...,24.9,3,41.913390,-8.563963,"Paredes de Coura, Viana do Castelo, Portugal",359.3,84.9,18.0,200.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Município,Regiao Autonoma da Madeira,Ribeira Brava,1589,8652,2156,12.8,69.8,17.4,189.4,...,28.8,5,32.670092,-17.065040,"Ribeira Brava, Madeira, Portugal",981.0,1204.1,15.0,153.0,5.0
304,Município,Regiao Autonoma da Madeira,Santa Cruz,6922,33094,5449,15.2,72.8,12.0,559.1,...,15.2,9,32.698335,-16.801762,"Santa Cruz, Madeira, Portugal",961.5,1187.1,16.0,57.0,2.0
305,Município,Regiao Autonoma da Madeira,Santana,644,4356,1680,9.6,65.2,25.2,70.0,...,12.4,3,32.804907,-16.881301,"Santana, Madeira, Portugal",958.2,1181.9,15.0,217.0,8.0
306,Município,Regiao Autonoma da Madeira,Sao Vicente,494,3331,1304,9.6,64.9,25.4,65.0,...,17.2,2,32.802276,-17.047240,"São Vicente, Madeira, Portugal",969.4,1191.4,21.0,253.0,10.0


In [20]:
file_name = "{}{}".format('df_municipios_ranking_geral',".csv")
df_municipios.to_csv(file_name, sep=';', quotechar='"',decimal=',', encoding='utf-8')
